# <center>Data Cleaning with PySpark</center>

## Spark Schemas
A primary function of data cleaning is to verify all data is in the expected format.  In Spark, datasets can be validated with schemas, which serve as a blueprint for how to structure data in a DataFrame (e.g. define the number and types of columns). A schema defines the column names, data types, and whether or not null values are allowed for each column.

<u>Pros</u>:
- **Filter garbage data**: A schema allows Spark to filter out data that does not conform during read, although it may be best to investigate these data further. 
- **Improved read performance**: Spark does not have to infer the type of each column since this information is provided by the schema.

<u>Cons</u>:
- **Loss of data**: When you define a schema, you are specifying the exact data types and column names for your DataFrame. This means that if you have mixed data types or if you have extra columns, this data will be lost during the import process. It may be best to deal with these issues before defining a schema.
- **Schema changes can be difficult to manage**: If the data changes over time or if you need to add or remove columns, you may need to update your schema accordingly.


The schema is defined using the `StructType` and `StructField` classes from the `pyspark.sql.types` module. The `StructType` class is used to define the overall structure of the schema, while the `StructField` class is used to define the individual fields or columns within the schema.

In [1]:
from pyspark.sql import SparkSession
from  pyspark.sql.types import *

# Define the schema
schema = StructType([
    StructField("Id", IntegerType(), False),
    StructField("Name", StringType(), True),
    StructField("Sex", StringType(), True),
    StructField("Age", FloatType(), True),
    StructField("Cabin", StringType(), True)
])

# Create session and import data
spark = SparkSession.builder.getOrCreate()
df = spark.read.csv("C:\\Users\\ihadjimpalasis\\Desktop\\Learning\\Data Engineer Datacamp\\titanic_subset.csv", header=True, schema=schema)
df.show(5)

+---+--------------------+------+----+-----+
| Id|                Name|   Sex| Age|Cabin|
+---+--------------------+------+----+-----+
|  0|Braund, Mr. Owen ...|  male|22.0| null|
|  1|Cumings, Mrs. Joh...|female|38.0|  C85|
|  2|Heikkinen, Miss. ...|female|26.0| null|
|  3|Futrelle, Mrs. Ja...|female|35.0| C123|
|  4|Allen, Mr. Willia...|  male|35.0| null|
+---+--------------------+------+----+-----+
only showing top 5 rows



The order of the `StructField` constructors in the `StructType` object determines the order of the columns in the resulting DataFrame. In each constructor we defined a column, its type, and set the nullability flag to either `True` or `False`. The nullability flag indicates if null values are allowed in the column, and whether invalid values will be converted to null. When it is set to `True`, all values that do not conform with the column's data type will be converted to null. An error will be raised if an invalid value is encountered and the flag is set to `False`. 

## Spark DataFrame Operations

Some common methods for carrying out operations on a DataFrame include:

- select(): It is used to select one or more columns from a DataFrame. <br>
Example: `df.select("col1", df["col2"])`

- filter() / where(): It is used to filter rows of a DataFrame based on one or more conditions. <br>
Example: `df.filter(df["col1"] > 10)`

- withColumn(): It is used to modify an existing column or add a new one based on the values of one or more existing columns. <br>
Example: `df.withColumn("new_col", df["col1"] + df["col2"])`

- groupBy(): It is used to group a DataFrame by one or more columns and perform aggregate functions on each group. <br>
Example: `df.groupBy("col1").agg(avg("col2"))`

- orderBy(): It is used to sort a DataFrame by one or more columns. You can specify the column names and the sort order (ascending or descending). <br>
Example: `df.orderBy(df["col1"].asc(), df["col2"].desc())`

- distinct(): It is used to return the distinct values of a DataFrame. You can either specify the column names or use no arguments to return distinct rows. <br>
Example: `df.distinct()`

- join(): It is used to join two DataFrames based on one or more columns. You can specify the join type (inner, left, right, or full outer) and the join columns. <br>
Example: `df1.join(df2, ["col1", "col2"], "inner")`

Note that Spark DataFrames are immutable, meaning that changes do not occur inplace, but need to be stored in a DataFrame variable. To demonstrate, let's use the `.withColumn()` method to extract the numeric part of the "Cabin" column into a new column called "Cabin_Num" and the alphabetic part into another column called "Cabin_Lett". For this, we will need to use the sql function `regexp_extract()` which allows us to extract a subset from a string using a regular expression.

In [6]:
from pyspark.sql.functions import regexp_extract, col

df = (df
      .withColumn("Cabin_Num", regexp_extract(col("Cabin"), r"\d+", 0)) # col("Cabin") is an alternative to df["Cabin"]
      .withColumn("Cabin_Num", col("Cabin_Num").cast(IntegerType()))
      .withColumn("Cabin_Lett", regexp_extract(col("Cabin"), r"[A-Za-z]+", 0))
      )

df.show(5)

+---+--------------------+------+----+-----+---------+----------+
| Id|                Name|   Sex| Age|Cabin|Cabin_Num|Cabin_Lett|
+---+--------------------+------+----+-----+---------+----------+
|  0|Braund, Mr. Owen ...|  male|22.0| null|     null|      null|
|  1|Cumings, Mrs. Joh...|female|38.0|  C85|       85|         C|
|  2|Heikkinen, Miss. ...|female|26.0| null|     null|      null|
|  3|Futrelle, Mrs. Ja...|female|35.0| C123|      123|         C|
|  4|Allen, Mr. Willia...|  male|35.0| null|     null|      null|
+---+--------------------+------+----+-----+---------+----------+
only showing top 5 rows



The `regexp_extract` function is used to extract substrings from a string column based on a regular expression pattern. It has three mandatory parameters: str, patter, idx.

- **str**: The input string column that you want to extract substrings from.
- **pattern**: The regular expression pattern that you want to match against the input string. 
- **idx**: The index of the capturing group in the regular expression pattern that you want to extract. 

When using capturing groups, idx should be set to the index of the capturing group that contains the desired substring. For example, if the regular expression pattern is `(\\w+):(\\d+)` and we want to extract the second group (i.e., the digits after the colon), we would use idx = 2. If we want to extract the entire match like in our example above, we would use idx = 0.

We have seen how `.withColumn()` works, now let's investigate select and filter.

In [7]:
df_new = (df
          .select("Name","Sex","Age","Cabin","Cabin_Num","Cabin_Lett")
          .filter(df["Cabin"].isNotNull()) 
          .drop("Cabin")
          )

df_new.show(5)

+--------------------+------+----+---------+----------+
|                Name|   Sex| Age|Cabin_Num|Cabin_Lett|
+--------------------+------+----+---------+----------+
|Cumings, Mrs. Joh...|female|38.0|       85|         C|
|Futrelle, Mrs. Ja...|female|35.0|      123|         C|
|McCarthy, Mr. Tim...|  male|54.0|       46|         E|
|Sandstrom, Miss. ...|female| 4.0|        6|         G|
|Bonnell, Miss. El...|female|58.0|      103|         C|
+--------------------+------+----+---------+----------+
only showing top 5 rows



## SQL Functions

The `pyspark.sql.functions` module provides a collection of built-in functions for working with structured data. These functions can be used for tasks such as data cleaning, data transformation, and data aggregation.

Some of the commonly used functions in `pyspark.sql.functions` are:
- col: This function is used to create a Column object that represents a column in a DataFrame.
- lit: This function is used to create a new column with a literal value. Useful when you want to add a constant value to a DataFrame.
- concat: This function is used to concatenate two or more columns in a DataFrame.
- when: This function is used to conditionally evaluate a column based on a given condition.
- regexp_replace: This function is used to replace substrings that match a regular expression pattern with a given replacement string.
- avg, sum, min, max: These functions are used to compute basic aggregation functions on a DataFrame.
- date_add, date_sub: These functions are used to add or subtract a specified number of days from a date column.
- explode: This function is used to transform a column of arrays or maps into multiple rows.

To avoid importing functions one-by-one, we can import all of them as `F` as shown in the example below using `split()`:

In [9]:
import pyspark.sql.functions as F

df_new = (df_new
            .withColumn("Name", F.split(col("Name"), ", "))
            .withColumn("Last_Name", col("Name")[0])
            .withColumn("First_Name", col("Name")[1])
            .drop("Name"))

df_new.show(5)

+------+----+---------+----------+---------+--------------------+
|   Sex| Age|Cabin_Num|Cabin_Lett|Last_Name|          First_Name|
+------+----+---------+----------+---------+--------------------+
|female|38.0|       85|         C|  Cumings|Mrs. John Bradley...|
|female|35.0|      123|         C| Futrelle|Mrs. Jacques Heat...|
|  male|54.0|       46|         E| McCarthy|       Mr. Timothy J|
|female| 4.0|        6|         G|Sandstrom|Miss. Marguerite Rut|
|female|58.0|      103|         C|  Bonnell|     Miss. Elizabeth|
+------+----+---------+----------+---------+--------------------+
only showing top 5 rows

